In [21]:
from pyxdf import match_streaminfos, resolve_streams
from mnelab.io.xdf import read_raw_xdf
import mne
import mnelab
import numpy as np
import matplotlib.pyplot as plt
import os.path
import pandas as pd
import scipy
import pickle

In [30]:
reject_criteria = dict(eeg=100e-6)
def read_file(raw):
    raw.filter(2, 20)
    raw2 =raw.copy().resample(150)
    # ica = mne.preprocessing.ICA(n_components = 4, random_state = 0)
    # ica.fit(raw2)
    # raw2 = ica.apply(raw2,exclude = [0])
    events, _ = mne.events_from_annotations(raw2)
    # "Non distraction + congruent":6, "distraction + congruent" :4 , "Non distraction + incongruent":7, "distraction + incongruent" :5
    event_ids = {"Non distraction + incongruent":7, "distraction + incongruent" :5}
    epochs = mne.Epochs(raw2, events,event_id = event_ids,  preload=True,tmin = -0.5, tmax=1 , baseline=(None, 0),  reject=reject_criteria)
    labels=epochs.events[:,-1]
    features=epochs.get_data()
    labels[labels == 5] = 1 # distracted
    #labels[labels == 5] = 1 # distracted
    labels[labels == 7 ] = 0 # nd
    #labels[labels == 7] = 0 # nd

    xx= np.array(features)
        
    NormalizedAll = xx - xx.min();
    NormalizedAll = NormalizedAll / xx.max();
    NormalizedAll=NormalizedAll.reshape(features.shape[0], 4, features.shape[2])

    mat = np.zeros((features.shape[0],4,4))
    for i in range (0,features.shape[0]):
        Pearson_matrix = np.corrcoef(NormalizedAll[i,:,:]);
        abs_Pearson_matrix =  abs(Pearson_matrix)
        mat[i,:,:] = abs_Pearson_matrix -  np.eye(4, 4);

    # epoch_incongruent_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + congruent"],  preload=True,tmin = -0.5, tmax=0.8 , baseline=(None, 0))
    # epoch_incongruent_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + congruent"],  preload=True,tmin = -0.5, tmax=0.8, baseline=(None, 0))

    # epoch_incongruent_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    # epoch_incongruent_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5, baseline=(None, 0))

    # epoch_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + congruent", "distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    # epoch_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + congruent", "distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    
    return mat,labels

Event ids are, 41 =  D + congruent, 42 =  D + incongruent,  51 =  ND + congruent, 52 =  ND + incongruent

In [24]:
mne.set_log_level(verbose='WARNING', return_old_level=False, add_frames=None)

In [39]:
graphs, labels, groups=[],[],[]
for pnum in range(1,21):
    paraticipant = "P"+ str(pnum)
    print(pnum) 

    path ='./'+ paraticipant +'/'
    raw = None 
    for num in range(1,5):
        fileNamei = paraticipant +'_b'  + str(num) + ".xdf"
        path_addr = path + fileNamei
        if (os.path.exists(path_addr)):
            streams = resolve_streams(path_addr)
            stream_id = match_streaminfos(streams, [{"type": "EEG"}])[0]
            rawi = read_raw_xdf(path_addr, stream_ids=[stream_id]).resample(250)
            
            if (raw is not None):
                mne.io.concatenate_raws([raw, rawi], on_mismatch="ignore")
            else:
                raw = rawi  

            if (pnum == 13) or (pnum == 19):
                raw = None      

    if (raw is not None):
        graphs_sess1,label_sess1= read_file(raw)
        graphs.append(graphs)
        labels.append(label_sess1)
        groups.append([pnum]*(len(label_sess1)))

Stream 2: Calculated effective sampling rate 473.7025 Hz is different from specified rate 256.0000 Hz.


1


Stream 2: Calculated effective sampling rate 455.8237 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 475.2557 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 459.6645 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 476.0124 Hz is different from specified rate 256.0000 Hz.


2


Stream 2: Calculated effective sampling rate 390.8616 Hz is different from specified rate 256.0000 Hz.
/var/folders/x4/6xjb_tfs6vdg6llm0_l4vjz00000gn/T/ipykernel_1069/165952213.py:17: RuntimeWarning: Omitted 8 annotation(s) that were outside data range.
  mne.io.concatenate_raws([raw, rawi], on_mismatch="ignore")
Stream 2: Calculated effective sampling rate 454.4369 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 464.3009 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 467.3554 Hz is different from specified rate 256.0000 Hz.


...

Stream 2: Calculated effective sampling rate 482.7663 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 475.3240 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 456.3403 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 469.7415 Hz is different from specified rate 256.0000 Hz.


19


Stream 2: Calculated effective sampling rate 467.5118 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 459.9489 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 470.8543 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 485.2200 Hz is different from specified rate 256.0000 Hz.


20


Stream 2: Calculated effective sampling rate 471.5687 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 439.5069 Hz is different from specified rate 256.0000 Hz.
/var/folders/x4/6xjb_tfs6vdg6llm0_l4vjz00000gn/T/ipykernel_1069/165952213.py:17: RuntimeWarning: Omitted 5 annotation(s) that were outside data range.
  mne.io.concatenate_raws([raw, rawi], on_mismatch="ignore")
Stream 2: Calculated effective sampling rate 475.3133 Hz is different from specified rate 256.0000 Hz.


In [ ]:
# import numpy as np
# data_array=np.concatenate(features)
# labels=np.concatenate(labels)
# groups=np.concatenate(groups)

# features.shape,labels.shape,groups.shape

In [ ]:
graph_array=np.vstack(graphs)

In [41]:

label_array=np.hstack(labels)
group_array=np.hstack(groups)
print(graph_array.shape,label_array.shape,group_array.shape)

NameError: name 'graph_array' is not defined

In [ ]:
with open('incongruent_features_150hz.pickle', 'wb') as output:
    pickle.dump(data_array, output)


with open('incongruent_label_150hzs.pickle', 'wb') as output:
    pickle.dump(label_array, output)


with open('incongruent_group_150hzs.pickle', 'wb') as output:
    pickle.dump(group_array, output)    




In [ ]:
np.unique(group_array, axis=0)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 14.,
       15., 16., 17., 18., 20.])

LR

In [ ]:
from scipy import stats
def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)


def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [ ]:
from tqdm import tqdm_notebook
features=[]
for data in tqdm_notebook(data_array):
    features.append(concatenate_features(data))
features=np.array(features)
features.shape

C:\Users\erang\AppData\Local\Temp\ipykernel_12504\3783067424.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data in tqdm_notebook(data_array):


  0%|          | 0/878 [00:00<?, ?it/s]

(878, 52)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
clf=LogisticRegression()
# gkf=GroupKFold(n_splits=5)
logo = LeaveOneGroupOut()
# logo.split(data_array, label_array, groups=group_array):
param_grid = {'classifier__C': [0.001, 0.005, 0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(features,label_array,groups=group_array)


KeyboardInterrupt



In [ ]:
from mne_features.feature_extraction import FeatureExtractor
selected_funcs = ['pow_freq_bands', 'line_length', 'kurtosis', 'ptp_amp', 'skewness']
# fe = FeatureExtractor(sfreq=256, selected_funcs=selected_funcs)

pipe = Pipeline([('fe', FeatureExtractor(sfreq=256,
                                         selected_funcs=selected_funcs)),
                                           ('scaler', StandardScaler()),
                 ('classifier', clf)])
logo = LeaveOneGroupOut()
param_grid = {'classifier__C': [0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]}
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(data_array,label_array,groups=group_array)




c:\Users\erang\.conda\envs\env\lib\site-packages\mne_features\utils.py:21: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit()


In [ ]:
from sklearn.svm import SVC
param_grid = {'classifier__C': [0.1, 1, 10, 100, 1000], 
              'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'classifier__kernel': ['rbf']} 
selected_funcs = ['pow_freq_bands', 'line_length', 'kurtosis', 'ptp_amp', 'skewness']

pipe = Pipeline([('fe', FeatureExtractor(sfreq=256,
                                         selected_funcs=selected_funcs)),
                                           ('scaler', StandardScaler()),
                 ('classifier', SVC())])
logo = LeaveOneGroupOut()
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(data_array,label_array,groups=group_array)

GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=Pipeline(steps=[('fe',
                                        FeatureExtractor(selected_funcs=['pow_freq_bands',
                                                                         'line_length',
                                                                         'kurtosis',
                                                                         'ptp_amp',
                                                                         'skewness'],
                                                         sfreq=256)),
                                       ('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             n_jobs=16,
             param_grid={'classifier__C': [0.1, 1, 10, 100, 1000],
                         'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'classifier__kernel': ['rbf']})

In [ ]:
X.shape

(1546, 20)

In [ ]:
gscv.best_score_

0.4972497065171553

In [ ]:
from sklearn.svm import SVC
param_grid = {'classifier__C': [0.1, 1, 10, 100, 1000], 
              'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'classifier__kernel': ['rbf']} 
selected_funcs = ['pow_freq_bands']

pipe = Pipeline([('fe', FeatureExtractor(sfreq=256,
                                         selected_funcs=selected_funcs)),
                                           ('scaler', StandardScaler()),
                 ('classifier', SVC())])
logo = LeaveOneGroupOut()
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(data_array,label_array,groups=group_array)

In [ ]:
gscv.best_params_

{'classifier__C': 0.1, 'classifier__gamma': 1, 'classifier__kernel': 'rbf'}

In [ ]:
# np.unique(labels) #7 left 8 right

In [ ]:
unique, counts = np.unique(groups, return_counts=True)
unique, counts

c:\Users\erang\.conda\envs\env\lib\site-packages\numpy\lib\arraysetops.py:272: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


(array([list([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
        list([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        list([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
        list([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [ ]:
# gscv.best_score_
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()


CNN

In [ ]:
# data_array=np.vstack(data_array)
# label_array=np.hstack(label_array)
# group_array=np.hstack(group_array)
data_array_dl=np.moveaxis(data_array,1,2)

print(data_array.shape)

print(data_array_dl.shape,label_array.shape,group_array.shape)

(1546, 4, 334)
(1546, 334, 4) (1546,) (1546,)


In [ ]:
reshaped_data =  np.reshape(data_array, (4, 167,1553))

print(data_array.shape)

ValueError: cannot reshape array of size 2065456 into shape (4,167,1553)

In [ ]:

# accuracy=[]
# for train_index, val_index in logo.split(data_array, label_array, groups=group_array):
#     train_features,train_labels=data_array[train_index],label_array[train_index]
#     val_features,val_labels=data_array[val_index],label_array[val_index]
#     scaler=StandardScaler()
#     train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
#     val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
#     model=EEGNetModel
#     model.fit(train_features,train_labels,epochs=20,batch_size=96,validation_data=(val_features,val_labels))
#     accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/20
16/16 [==============================] - 1s 30ms/step - loss: 0.6804 - accuracy: 0.4928 - val_loss: 0.7052 - val_accuracy: 0.5000
Epoch 2/20
16/16 [==============================] - 1s 31ms/step - loss: 0.6820 - accuracy: 0.4928 - val_loss: 0.7050 - val_accuracy: 0.5000
Epoch 3/20
16/16 [==============================] - 1s 35ms/step - loss: 0.6775 - accuracy: 0.4928 - val_loss: 0.7050 - val_accuracy: 0.5000
Epoch 4/20
16/16 [==============================] - 1s 33ms/step - loss: 0.6838 - accuracy: 0.4928 - val_loss: 0.7058 - val_accuracy: 0.5000
Epoch 5/20
16/16 [==============================] - 1s 34ms/step - loss: 0.6803 - accuracy: 0.4928 - val_loss: 0.7067 - val_accuracy: 0.5000
Epoch 6/20
16/16 [==============================] - 0s 30ms/step - loss: 0.6795 - accuracy: 0.4928 - val_loss: 0.7063 - val_accuracy: 0.5000
Epoch 7/20
16/16 [==============================] - 0s 29ms/step - loss: 0.6793 - accuracy: 0.4928 - val_loss: 0.7062 - val_accuracy: 0.5000
Epoch 8/20
16

https://github.com/JodieAsh/Real-Time-EEG-Classification-with-Muse/blob/master/Build%26TestClassifier.py
https://link.springer.com/chapter/10.1007/978-3-030-29933-0_37
https://github.com/jordan-bird/eeg-feature-generation